<a href="https://colab.research.google.com/github/aniketsaha2310/MoodTune/blob/main/xception_emotion_classifier_h5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import PIL
import PIL.Image
import tensorflow as tf
import warnings

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy


In [4]:
# Define data paths (using your provided paths)
train_path = "/content/drive/MyDrive/MoodTune-main/Dataset/train"
test_path = "/content/drive/MyDrive/MoodTune-main/Dataset/test"

In [7]:
# Image dimensions (adjust if necessary)
img_width, img_height = 224, 224

In [8]:
# Data augmentation for training (optional)
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)  # Rescale only for testing


In [9]:
# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical'  # Assuming categorical labels
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical'  # Assuming categorical labels
)


Found 41275 images belonging to 4 classes.
Found 5140 images belonging to 4 classes.


In [12]:
# training images are organized in subdirectories representing classes
import os

train_dir = "/content/drive/MyDrive/MoodTune-main/Dataset/train"
class_names = os.listdir(train_dir)

# Sort class names alphabetically (optional)
class_names.sort()

print(class_names)


['angry', 'happy', 'neutral', 'sad']


In [14]:
# 1. Build the Model
model = Sequential()

In [15]:
# Load pre-trained Xception model (include_top=False for transfer learning)
base_model = Xception(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))


83683744/83683744 [==============================] - 0s 0us/step


In [16]:
# Add the pre-trained model to the sequential model
model.add(base_model)


In [17]:
# Add additional layers on top
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

In [18]:
# 2. Summarize the Model (Optional)
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 256)               25690368  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 46552876 (177.59 MB)
Trainable params: 46498348 (177.38 MB)
Non-trainable params: 54528 (213.00 KB)
_________________________________________________________________
None


In [19]:
# 3. Train the Model with Data Generators
model.compile(optimizer=Adam(learning_rate=0.001),
              loss=CategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.fit(train_generator,
          epochs=5,
          validation_data=test_generator)

Epoch 1/5
  20/1290 [..............................] - ETA: 9:53:01 - loss: 1.6373 - accuracy: 0.2984

In [ ]:
from tensorflow.keras.models import save_model


In [ ]:
# Saved the model
save_model(model, 'xception_emotion_classifier.h5')
